In [ ]:
PATH = os.getcwd()
PATH_DATA = os.path.join(PATH, 'data')
PATH_DATA_CATEGORIES = os.path.join(PATH_DATA, 'categories')
PATH_DATA_MODEL = os.path.join(PATH_DATA, 'model_h5')
PATH_DATA_CATEGORIES_DICTIONARY = os.path.join(PATH_DATA, 'categories.dict')

IMAGE_SIZE = (32, 32)
TRAIN_SIZE = 0.9
RANDOM_STATE = 420

KERNEL_SIZE1 = 32
KERNEL_SIZE2 = 64
STRIDES = (3, 3)
PADDING = 'same'
ACTIVATION1 = 'relu'
ACTIVATION2 = 'softmax'
POOL_SIZE = (2, 2)
DROPOUT = 0.25
DENSE = 512

LR = 0.0001
DECAY = 1e-6

LOSS = 'categorical_crossentropy'
METRICS = ['accuracy']

BATCH_SIZE = 32
EPOCHS = 30

In [ ]:
import argparse
import pickle
import os
import numpy as np
import cv2 as cv
from PIL import Image, ImageFilter
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation
from keras import models
from keras.engine.functional import Functional
from keras.models import Sequential, load_model
from keras.utils import to_categorical
from keras import optimizers
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation
from tensorflow.keras.engine.functional import Functional
from keras.optimizers import Adam

In [ ]:
from PIL import Image

#przejscie przez elementy w PATH_DATA_CATEGORIES
for dir_name in os.listdir(PATH_DATA_CATEGORIES):
  #sprawdzenie czy nie zaczyna sie od . -> potrzeba bo .DS_store na mac
    if not dir_name.startswith('.'):
        dir_path = os.path.join(PATH_DATA_CATEGORIES, dir_name)
        print('[+] Augmenting images in {}'.format(dir_path))
        for image_name in os.listdir(dir_path):
            image_path = os.path.join(dir_path, image_name)
            img, ext = os.path.splitext(image_path)
            shape = np.array(Image.open(image_path).convert('RGB')).shape #kształt obrazu

            #zmiana do rgb bo jpeg
            Image.open(image_path).convert('RGB').crop((15, 45, shape[1] - 45, shape[1] - 15)).save(
                '{}_CROPPED{}'.format(img, ext))
            for rot in range(15, 360, 15):
                Image.open(image_path).convert('RGB').rotate(rot).save('{}_R{}{}'.format(img, rot, ext))
                Image.open(image_path).convert('RGB').transpose(Image.FLIP_LEFT_RIGHT).save('{}_LR{}'.format(img, ext))
                Image.open(image_path).convert('RGB').transpose(Image.FLIP_TOP_BOTTOM).save('{}_TB{}'.format(img, ext))
                Image.open(image_path).convert('RGB').filter(ImageFilter.GaussianBlur(3)).save('{}_BLUR{}'.format(img, ext))
                Image.open(image_path).convert('RGB').filter(ImageFilter.SMOOTH).save('{}_SMOOTH{}'.format(img, ext))


[+] Augmenting images in /content/data/categories/ksiazka


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


[+] Augmenting images in /content/data/categories/kubek
[+] Augmenting images in /content/data/categories/klawiatura


In [ ]:
categories_dictionary = dict() #mapowanie indeksów kategorii na nazwy kategorii.
images_list = list()
categories_list = list()
i = 0
for dir_name in os.listdir(PATH_DATA_CATEGORIES):
  if not dir_name.startswith('.'):
                categories_dictionary.update({i: dir_name})
                dir_path = os.path.join(PATH_DATA_CATEGORIES, dir_name)
                print('[+] Reading images in {}'.format(dir_path))
                for image_name in os.listdir(dir_path):
                    image_path = os.path.join(dir_path, image_name)
                    image = Image.open(image_path)
                    image = image.resize(IMAGE_SIZE, resample=Image.BILINEAR)
                    image_array = np.array(image)
                    if image_array.shape != (32, 32, 3):
                        continue  # Pomijaj obrazy, które nie mają oczekiwanego rozmiaru
                    images_list.append(image_array)
                    categories_list.append(i)
                i += 1

print('[+] Saving categories dictionary to {}'.format(PATH_DATA_CATEGORIES_DICTIONARY))
with open(PATH_DATA_CATEGORIES_DICTIONARY, 'wb') as dictionary_file: pickle.dump(categories_dictionary, dictionary_file)

images_array = np.asarray(images_list)
categories_array = np.asarray(categories_list)

x_train, x_test, y_train, y_test = train_test_split(images_array, categories_array,
                                                            train_size=TRAIN_SIZE,
                                                            random_state=RANDOM_STATE)

y_train = to_categorical(y_train, len(categories_dictionary.keys()))
y_test = to_categorical(y_test, len(categories_dictionary.keys()))

#model i warstwy konwolucyjne
print('Model')
model = Sequential()
model.add(
            Conv2D(KERNEL_SIZE1, STRIDES, padding=PADDING,
                   input_shape=images_array.shape[1:]))
model.add(Activation(ACTIVATION1))
model.add(Conv2D(KERNEL_SIZE1, STRIDES))
model.add(Activation(ACTIVATION1))
model.add(MaxPooling2D(pool_size=POOL_SIZE))
model.add(Dropout(DROPOUT))

model.add(Conv2D(KERNEL_SIZE2, STRIDES, padding=PADDING))
model.add(Activation(ACTIVATION1))
model.add(Conv2D(KERNEL_SIZE2, STRIDES))
model.add(Activation(ACTIVATION1))
model.add(MaxPooling2D(pool_size=POOL_SIZE))
model.add(Dropout(DROPOUT))

model.add(Flatten())
model.add(Dense(DENSE))
model.add(Activation(ACTIVATION1))
model.add(Dropout(DROPOUT))
model.add(Dense(len(categories_dictionary.keys())))
model.add(Activation(ACTIVATION2))

model.summary()

print('[+] Compiling and fitting model')

#learning_rate_decay nie jest dostępny w optymalizatorze RMSprop w Keras.
optimizer = optimizers.RMSprop(lr=LR, decay=DECAY)

model.compile(loss=LOSS, optimizer=optimizer, metrics=METRICS)

model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS,
                  validation_data=(x_test, y_test), shuffle=True)

print('[+] Saving model to {}'.format(PATH_DATA_MODEL))
model.save(PATH_DATA_MODEL)
test_loss, test_acc = model.evaluate(x_test, y_test)
print('[+] Loss: {}, Acc: {}'.format(test_loss, test_acc))

[+] Reading images in /content/data/categories/ksiazka
[+] Reading images in /content/data/categories/kubek
[+] Reading images in /content/data/categories/klawiatura
[+] Saving categories dictionary to /content/data/categories.dict
Model
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 activation_12 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 conv2d_9 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 activation_13 (Activation)  (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 15, 15, 32)       0         
 2D)          

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


1/1 [==============================] - 2s 2s/step - loss: 16.4197 - accuracy: 0.3846 - val_loss: 16.9142 - val_accuracy: 0.6667
Epoch 2/30
1/1 [==============================] - 1s 592ms/step - loss: 28.5524 - accuracy: 0.3846 - val_loss: 8.3809 - val_accuracy: 0.0000e+00
Epoch 3/30
1/1 [==============================] - 1s 647ms/step - loss: 18.9895 - accuracy: 0.3846 - val_loss: 13.6837 - val_accuracy: 0.6667
Epoch 4/30
1/1 [==============================] - 1s 543ms/step - loss: 21.3460 - accuracy: 0.4231 - val_loss: 7.0769 - val_accuracy: 0.3333
Epoch 5/30
1/1 [==============================] - 1s 586ms/step - loss: 8.2255 - accuracy: 0.5000 - val_loss: 4.7157 - val_accuracy: 0.6667
Epoch 6/30
1/1 [==============================] - 1s 519ms/step - loss: 6.5966 - accuracy: 0.5000 - val_loss: 4.7121 - val_accuracy: 0.3333
Epoch 7/30
1/1 [==============================] - 0s 453ms/step - loss: 13.4938 - accuracy: 0.3846 - val_loss: 8.0298 - val_accuracy: 0.3333
Epoch 8/30
1/1 [=======

1/1 [==============================] - 0s 43ms/step - loss: 1.2586 - accuracy: 0.6667
[+] Loss: 1.2586236000061035, Acc: 0.6666666865348816


In [15]:
categories_dictionary = dict()
images_list = list()
categories_list = list()
i = 0
for dir_name in os.listdir(PATH_DATA_CATEGORIES):
  if not dir_name.startswith('.'):
                categories_dictionary.update({i: dir_name})
                dir_path = os.path.join(PATH_DATA_CATEGORIES, dir_name)
                print('[+] Reading images in {}'.format(dir_path))
                for image_name in os.listdir(dir_path):
                    image_path = os.path.join(dir_path, image_name)
                    image = Image.open(image_path)
                    image = image.resize(IMAGE_SIZE, resample=Image.BILINEAR)
                    image_array = np.array(image)
                    if image_array.shape != (32, 32, 3):
                        continue  # Pomijaj obrazy, które nie mają oczekiwanego rozmiaru
                    images_list.append(image_array)
                    categories_list.append(i)
                i += 1

print('[+] Saving categories dictionary to {}'.format(PATH_DATA_CATEGORIES_DICTIONARY))
with open(PATH_DATA_CATEGORIES_DICTIONARY, 'wb') as dictionary_file: pickle.dump(categories_dictionary, dictionary_file)

images_array = np.asarray(images_list)
categories_array = np.asarray(categories_list)

x_train, x_test, y_train, y_test = train_test_split(images_array, categories_array,
                                                            train_size=TRAIN_SIZE,
                                                            random_state=RANDOM_STATE)

y_train = to_categorical(y_train, len(categories_dictionary.keys()))
y_test = to_categorical(y_test, len(categories_dictionary.keys()))

print('Model')
model = Sequential()
model.add(
            Conv2D(KERNEL_SIZE1, STRIDES, padding=PADDING,
                   input_shape=images_array.shape[1:]))
model.add(Activation(ACTIVATION1))
model.add(Conv2D(KERNEL_SIZE1, STRIDES))
model.add(Activation(ACTIVATION1))
model.add(MaxPooling2D(pool_size=POOL_SIZE))
model.add(Dropout(DROPOUT))

model.add(Conv2D(KERNEL_SIZE2, STRIDES, padding=PADDING))
model.add(Activation(ACTIVATION1))
model.add(Conv2D(KERNEL_SIZE2, STRIDES))
model.add(Activation(ACTIVATION1))
model.add(MaxPooling2D(pool_size=POOL_SIZE))
model.add(Dropout(DROPOUT))

model.add(Flatten())
model.add(Dense(DENSE))
model.add(Activation(ACTIVATION1))
model.add(Dropout(DROPOUT))
model.add(Dense(len(categories_dictionary.keys())))
model.add(Activation(ACTIVATION2))

model.summary()

print('[+] Compiling and fitting model')

optimizer = optimizers.RMSprop(lr=LR, decay=DECAY)

#optimizer = optimizers.RMSprop(lr=LR, decay=DECAY)

model.compile(loss=LOSS, optimizer=optimizer, metrics=METRICS)

model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS,
                  validation_data=(x_test, y_test), shuffle=True)

print('[+] Saving model to {}'.format(PATH_DATA_MODEL))
model.save(PATH_DATA_MODEL)
test_loss, test_acc = model.evaluate(x_test, y_test)
print('[+] Loss: {}, Acc: {}'.format(test_loss, test_acc))

[+] Reading images in /content/data/categories/ksiazka
[+] Reading images in /content/data/categories/kubek
[+] Reading images in /content/data/categories/klawiatura
[+] Saving categories dictionary to /content/data/categories.dict
Model
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 activation_18 (Activation)  (None, 32, 32, 32)        0         
                                                                 
 conv2d_13 (Conv2D)          (None, 30, 30, 32)        9248      
                                                                 
 activation_19 (Activation)  (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 15, 15, 32)       0         
 2D)          

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


25/25 [==============================] - 7s 227ms/step - loss: 14.5978 - accuracy: 0.3657 - val_loss: 1.7545 - val_accuracy: 0.5057
Epoch 2/30
25/25 [==============================] - 4s 159ms/step - loss: 5.8184 - accuracy: 0.3606 - val_loss: 1.3534 - val_accuracy: 0.4943
Epoch 3/30
25/25 [==============================] - 5s 183ms/step - loss: 2.9619 - accuracy: 0.4463 - val_loss: 1.0905 - val_accuracy: 0.5287
Epoch 4/30
25/25 [==============================] - 5s 194ms/step - loss: 1.8357 - accuracy: 0.5038 - val_loss: 0.9306 - val_accuracy: 0.5632
Epoch 5/30
25/25 [==============================] - 4s 161ms/step - loss: 1.3044 - accuracy: 0.5435 - val_loss: 0.7615 - val_accuracy: 0.6667
Epoch 6/30
25/25 [==============================] - 5s 195ms/step - loss: 1.0551 - accuracy: 0.5934 - val_loss: 0.6249 - val_accuracy: 0.7011
Epoch 7/30
25/25 [==============================] - 5s 180ms/step - loss: 0.8688 - accuracy: 0.6522 - val_loss: 0.5456 - val_accuracy: 0.7931
Epoch 8/30
25/25

3/3 [==============================] - 0s 35ms/step - loss: 0.1228 - accuracy: 0.9655
[+] Loss: 0.12276821583509445, Acc: 0.9655172228813171


In [ ]:
import shutil
import os

# Ścieżka do folderu, który chcesz usunąć
folder_path = "/content/data/categories/ksiazka"

# Usunięcie zawartości folderu
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path) or os.path.islink(file_path):
        os.unlink(file_path)
    elif os.path.isdir(file_path):
        shutil.rmtree(file_path)

# Usunięcie samego folderu
os.rmdir(folder_path)

# Potwierdzenie usunięcia folderu
print("Folder został usunięty wraz z zawartością.")


Folder został usunięty wraz z zawartością.
